<a href="https://colab.research.google.com/github/erikapaceep/NLP/blob/main/MLM_and_NSP_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLM and NSP summary

In [1]:
! pip install transformers
! pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import BertTokenizer, BertForPreTraining
import torch

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
text = ("After Abraham Lincoln won the November 1860 presidential [MASK] on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy. War broke out in "
        "April 1861 when secessionist forces [MASK] Fort Sumter in South "
        "Carolina, just over a month after Lincoln's inauguration.")

In [ ]:
text.split(" "), len(text.split(" "))

In [5]:
# tokenize the text
input = tokenizer(text, return_tensors='pt')

In [6]:
input

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,   103,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,   103,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
outputs = model(**input)

In [8]:
# form the output we have this prediciton logits and this sequence prediction logits
outputs.keys()

odict_keys(['prediction_logits', 'seq_relationship_logits'])

In [9]:
outputs.prediction_logits, outputs.prediction_logits.shape

(tensor([[[ -7.6192,  -7.5433,  -7.6124,  ...,  -6.7155,  -6.7375,  -4.6122],
          [-12.5489, -12.3772, -12.6500,  ..., -11.8644, -11.4446,  -9.1151],
          [ -6.2346,  -6.3590,  -5.9091,  ...,  -6.1258,  -6.2720,  -5.0268],
          ...,
          [ -2.2497,  -2.1352,  -2.1812,  ...,  -1.7201,  -1.2728,  -7.8301],
          [-14.2654, -14.3100, -14.2294,  ..., -11.4669, -11.7212, -10.3129],
          [-11.5071, -12.0389, -11.6046,  ..., -11.2875,  -9.1655,  -9.1732]]],
        grad_fn=<ViewBackward0>), torch.Size([1, 62, 30522]))

62 correspond to the:

In [10]:
input.input_ids.shape

torch.Size([1, 62])

the 30522 is the vocab size of bert: we see here a set of activations where each of those activations correspond to a specific token within the BERT vocabulary and each one of this item is a token: essentially the output predictions of the token based on the input prediciton.

Sequence relationship logit is the output from the NSP head. The output 2.825 is the activation on two sequences being consecutive or the is next classification. And the output -1.6897 is the non-next classification: the highest of the two will wind.

In [11]:
outputs.seq_relationship_logits

tensor([[ 2.8257, -1.6897]], grad_fn=<AddmmBackward0>)

----------------------------------------------


### MLM

MLM head, dive deeper and how we can take that and converet into token prediciton. **First thing we need to do is getting a mapping between our index values: the tokens ID and words form the model vocabulary.** we can the vocab from our tokenizer 

In [12]:
token2index = tokenizer.get_vocab()

In [ ]:
# we have a dictionary
token2index

In [28]:
token2index['hello']

7592

In [26]:
# we want to mapn from token ids to words: we invert the dict
indx2token = {value:key for key, value in token2index.items()}

In [35]:
len(indx2token)

30522

In [29]:
indx2token[7592]

'hello'

In [ ]:
# take the prediction logit and process it through a softmax function to get an index predicition for each value

In [57]:
outputs.prediction_logits[0][4].shape

torch.Size([30522])

In [58]:
outputs.prediction_logits.shape

torch.Size([1, 62, 30522])

In [63]:
# we take to softmax using the torch.nn.funcitonal, this is converting the logit into probability
softmax = torch.nn.functional.softmax(outputs.prediction_logits[0][4], dim=-1)

In [64]:
softmax.shape

torch.Size([30522])

In [65]:
#apply the argmax function
argmax = torch.argmax(softmax)
argmax

tensor(2180)

In [66]:
from jax.core import InDBIdx
indx2token[argmax.item()]

'won'

----------------------------------------

softmax for all the tokens

In [72]:
softmax = torch.nn.functional.softmax(outputs.prediction_logits[0], dim=0)

In [73]:
softmax.shape

torch.Size([62, 30522])

In [75]:
argmax = torch.argmax(softmax, dim=1)
# all these values are token ids 
argmax

tensor([28191,  2348,  8181, 16628,  2180,  3882,  2281,  7313,  4883, 27419,
         2006,  2010,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
         8914,  2163, 13520,  2037,  4336,  2013,  1996,  2406,  2000,  2433,
        28775, 18179, 16363,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
        18232,  2923,  2749,  4548,  3481,  7680,  5017,  2005,  2148,  3792,
        24901,  2074,  2058,  1037,  3204,  2077,  3946,  1005,  1055, 17331,
         1025, 25656])

In [71]:
outputs.prediction_logits[0].shape

torch.Size([62, 30522])

In [84]:
for i in argmax:
  print(indx2token[i.item()], end=' ')

##ecin although abraham lincolnshire won 1948 november 1860 presidential primaries on his anti - slavery platform , an initial seven tributary states declare their independence from the country to form ##ici confederacy ##yre war broke out in april 1861 when ##oya ##ist forces occupied fort sum ##mer for south carolina ##trip just over a month before grant ' s inauguration ; ##tson 

In [85]:
text

"After Abraham Lincoln won the November 1860 presidential [MASK] on an anti-slavery platform, an initial seven slave states declared their secession from the country to form the Confederacy. War broke out in April 1861 when secessionist forces [MASK] Fort Sumter in South Carolina, just over a month after Lincoln's inauguration."

the ### token is the BERT which uses word pieces encoding and not necessarly the word encoding and a lot of this closely match. 

-------------------------------------------------------------------------------

### NSP

NSP the output we receive is not particularly usefull because we only have one sentence in out input and we look we actually have more sentences. First we need to separate the two sentences

In [89]:
text1 = ("After Abraham Lincoln won the November 1860 presidential [MASK] on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy.") 
text2 = ("War broke out in April 1861 when secessionist forces [MASK] Fort Sumter in South "
        "Carolina, just over a month after Lincoln's inauguration.")

In [87]:
#before we had the following
input.input_ids

tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,   103,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,   103,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]])

In [90]:
input.input_ids.shape

torch.Size([1, 62])

In [88]:
input.token_type_ids

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [91]:
# How do we tokenize the two sentences together and we just pass both together in the format 
inputs = tokenizer(text1, text2, return_tensors='pt')

In [93]:
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,   103,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,   102,  2162,  3631,  2041,  1999,  2258,  6863,
          2043, 22965,  2923,  2749,   103,  3481,  7680,  3334,  1999,  2148,
          3792,  1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055,
         17331,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [94]:
inputs.token_type_ids
# unlike last time we have the 1s, they belong to the second sentence and the zeros are the sentence 1

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [95]:
inputs.input_ids
# they all look the same except that between text1 and text2 a token has been added 102 --> the tokenizer has done that to distinguish sentence A and B

tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,   103,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,   102,  2162,  3631,  2041,  1999,  2258,  6863,
          2043, 22965,  2923,  2749,   103,  3481,  7680,  3334,  1999,  2148,
          3792,  1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055,
         17331,  1012,   102]])

In [96]:
# let's check the original value
outputs.seq_relationship_logits

tensor([[ 2.8257, -1.6897]], grad_fn=<AddmmBackward0>)

In [97]:
outputs2 = model(**inputs)

In [98]:
#we do get different values: even though this won't change the classification
outputs2.seq_relationship_logits

tensor([[ 6.0843, -5.6813]], grad_fn=<AddmmBackward0>)

In [101]:
argmax = torch.argmax(outputs2.seq_relationship_logits)
'NotNext' if argmax.item() else 'IsNext' 

'IsNext'

Our predicted output: the model is correclty identifing the two sentences as a pair.